## **Segmentations and data extraction**

This notebook calls several scripts stored in its folder to segment full kidney, inner cavities and internal kidney area from LSFM images.

It also postprocesses those and the cyst masks produced with the model at the ./Networks folder, and extracts data from all the previously mentioned masks, and the glomeruli mask.

## **Segmentation and measurements of full kidney, inner cavities and kidney interior**

In [ ]:
#Run the process_fullKidneyMasks script to obtain the full kidney masks and the inner cavities masks (vessels) from the LSFM images of whole kidneys.
#This should be applied to each kidney image folder separately, specifying the input folder and the result folder.

from types import SimpleNamespace
from Kidneys_vessels.process_fullKidneyMasks import *

args = SimpleNamespace()

# Input parameters
args.strRad = 20
args.inputFolder = "E:/AAV para enfermedades renales/LSFM combined images/Full images/MacroSPIM3/R6LEC-Path_0.5_Lectine"
args.resultFolder = "E:/Github repositories/LSFM-processing-data/Full kidney and vessels segmentation/MacroSPIM3"
args.kidneyThres = 180
args.vessThres = 500

args.sliceToVis = 400
args.sliceBySliceThresholds = False
args.sliceBySliceErodeKidneyMask = True
args.radiusKidneyErosion = 50

args.processDimension = '3D'

# Applying the processing
processer = process_fullKidneyMasks(args)

#Choose which masks to obtain
processer.get_vessels_masks()
#processer.get_kidney_masks()


In [ ]:
#Postprocessing and measurements of the kidney masks

#Loop through all the kidney masks and apply slice by slice closing to remove holes from the kidney mask
#Then, generate a csv file with the centroid, area and inertia tensor eigenvalues of each kidney

from Cysts.morphoProcessingGPU import *
from measureConComps import *
import os
from glob import glob

mainInputFolder = 'E:/Github repositories/LSFM-processing-data/Full kidney and vessels segmentation'
mainOutputFolder = 'D:/0 - Thesis Data/Corrected kidney segmentations'

mainInputFolders = glob(mainInputFolder+'\*')

#For each of the main batches (MacroSPIM1, MacroSPIM2, MacroSPIM3)
for inputFolder in mainInputFolders[1:]:

    folderName = os.path.basename(inputFolder)
    print(f'Processing folder {folderName}')

    #Define the corresponding output folder
    outputFolder = f'{mainOutputFolder}/{folderName}'

    #Get a list with all kidney masks folders in the directory
    thingsInDir = glob(inputFolder+'\*')
    foldersInDir = [thing for thing in thingsInDir if 'kidneyMask' in os.path.basename(thing)]
    
    #For each kidney mask in the folder
    for i,kidneyPath in enumerate(foldersInDir):
        try:
            #Process the kidney mask
            kidneyName = os.path.basename(kidneyPath)
            print(f'Processing {kidneyName}')

            #Define the output folder
            resultMaskPath = f'{outputFolder}/{kidneyName}'

            #Close the kidney mask using 200 as radius
            morphoProcessingGPU(kidneyPath,resultMaskPath,strRad=200, operation = 'close',reScaleProp=0.25, imDimension = '2D', processDimension = '3D')

            #Extract the centroids, areas and inertia tensor eigenvalues and save them in a txt file
            measureConComps(resultMaskPath,outputFolder,reScaleProp=0.25,dimension='2D',inertiaTensEigen = True)
        except Exception as e:
            print('Moving pass error')
            print(repr(e))
            continue

In [ ]:
#Obtain kidney interior masks

#Loop through several full kidney masks and erode them in 3D using as radius for the structuring element
#half the smallest principal axis of the kidney mask, to approximately remove the cortex and keep only the kidney interior

from Cysts.morphoProcessingGPU import *
from glob import glob
import os
import pandas as pd
import numpy as np

mainInputFolder = 'D:/0 - Thesis Data/Corrected kidney segmentations'
mainOutputFolder = 'D:/0 - Thesis Data/Kidney interior segmentations'

mainInputFolders = glob(mainInputFolder+'\*')

#For each of the main batches (MacroSPIM1, MacroSPIM2, MacroSPIM3)
for inputFolder in mainInputFolders[1:]:

    folderName = os.path.basename(inputFolder)
    print(f'Processing folder {folderName}')

    #Define the corresponding output folder
    outputFolder = f'{mainOutputFolder}/{folderName}'

    #Get a list with all folders in the directory and another one with all files
    thingsInDir = glob(inputFolder+'\*')

    foldersInDir = [thing for thing in thingsInDir if os.path.isdir(thing)]
    filesInDir = [thing for thing in thingsInDir if os.path.isfile(thing)]

    #For each kidney mask in the folder
    for i,kidneyPath in enumerate(foldersInDir):
        try:
            #Process the kidney mask
            kidneyName = os.path.basename(kidneyPath)
            print(f'Processing {kidneyName}')

            #Get the corresponding data file (with the same order as the kidney mask)
            dataFilePath = filesInDir[i]

            #Define the output folder
            resultMaskPath = f'{outputFolder}/{kidneyName}'

            #Read the data file
            data = pd.read_csv(dataFilePath)

            #Get the principal axes lengths from the inertia tensor eigenvalues
            principalAxesLengths = [np.sqrt(data['inertia_tensor_eigvals-0']),np.sqrt(data['inertia_tensor_eigvals-1']),np.sqrt(data['inertia_tensor_eigvals-2'])]
            #Get the minimum principal axis length
            minPrincipalAxis = int(np.min(principalAxesLengths))
            strRad = minPrincipalAxis/2

            print(f'Minimum principal axis length: {minPrincipalAxis}')
            print(f'Erosion radius: {strRad}')

            #Erode the kidney mask using half the minimum principal axis length as radius
            morphoProcessingGPU(kidneyPath,resultMaskPath,strRad, operation = 'erode',reScaleProp=0.25, imDimension = '2D', processDimension = '3D')
        except Exception as e:
            print('Moving pass error')
            print(repr(e))
            continue

In [ ]:
#Create txt files measuring the volume of the inner regions

from Kidneys_vessels.measureVolumeBinary import *

from glob import glob
import os

mainInputFolder = 'D:/0 - Thesis Data/Kidney interior segmentations'

mainInputFolders = glob(mainInputFolder+'\*')

#For each of the main batches (MacroSPIM1, MacroSPIM2, MacroSPIM3)
for inputFolder in mainInputFolders:

    folderName = os.path.basename(inputFolder)
    print(f'Processing folder {folderName}')

    #Get a list with all folders in the directory and another one with all files
    thingsInDir = glob(inputFolder+'\*')

    foldersInDir = [thing for thing in thingsInDir if os.path.isdir(thing)]
    filesInDir = [thing for thing in thingsInDir if os.path.isfile(thing)]

    #For each kidney mask in the folder
    for i,kidneyPath in enumerate(foldersInDir):

        #Define the mask's name
        kidneyName = os.path.basename(kidneyPath)
        print(f'Processing {kidneyName}')

        #Measure the nonzero volume (in voxels)
        measureVolumeBinary(kidneyPath)

In [ ]:
#Extract the total volume of the cavity masks

from Kidneys_vessels.measureVolumeBinary import *
from glob import glob
import os

inputFolder = 'E:\Github repositories\LSFM-processing-data\Full kidney and vessels segmentation'

#Iterate through all the subfolders in the input folder
for folder in glob(inputFolder+'\*'):
    folderName = os.path.basename(folder)
    print(f'Processing folder {folderName}')


    #Get a list with all folders in the directory and another one with all files
    thingsInDir = glob(folder+'\*')

    #Take the folders with the cavity masks
    cavityMaskFolders = [thing for thing in thingsInDir if \
                           (os.path.isdir(thing) and 'vesselsMask' in os.path.basename(thing))]


    #Iterate through all the kidneys in the subfolder
    for i,mask in enumerate(cavityMaskFolders):

        if folderName == 'MacroSPIM2' and i==0 or i==1:
            continue

        maskName = os.path.basename(mask)

        print(f'Processing mask {maskName}')
       
        #Extract volumes
        print('Extracting volumes...')
        measureVolumeBinary(mask)   


In [ ]:
#Create txt files counting how many glomerulus centroids are in the inner region

from Inner_region.countCentroidsInner import *

mainPathInnerMask = 'D:/0 - Thesis Data/Kidney interior segmentations'
mainPathCentroids = 'E:\AAV para enfermedades renales\LSFM combined images\GlomerulusSegmentation' 

countCentroidsInner(mainPathInnerMask,mainPathCentroids,type = 'glomeruli')

## **Measurements of Glomeruli masks**

In [ ]:
#Loop through all the glomeruli to extract their centroids, volumes, surface areas and sphericities

from measureConComps import *
from glob import glob
import os

inputFolder = 'E:\AAV para enfermedades renales\LSFM combined images\GlomerulusSegmentation'

#Iterate through all the subfolders in the input folder
for folder in glob(inputFolder+'\*'):
    folderName = os.path.basename(folder)

    #Iterate through all the kidneys in the subfolder
    for kidney in glob(folder+'\*.tif'):
        kidneyName = os.path.basename(kidney)

        print(f'Processing kidney {kidneyName} from {folderName}')
       
        #Extract centroids and volumes
        print('Extracting centroids, volumes, surface area and sphericity...')
        measureConComps(kidney,folder,startingSlice=0,dimension='3D',typeOfMask='glomeruli',surfaces = True,saveSlices = False)   


## **Measurements of cyst masks** 

In [ ]:
#Loop through all the cysts masks and Apply GPU-accelelerated morphological opening to remove small particles + connect the components.

from Cysts.morphoProcessingGPU import *
from connectComponents import *
from glob import glob
import os

inputFolder = 'E:/Github repositories/LSFM-processing-data/Cysts segmentation'
outputFolder = 'D:/0 - Thesis Data/Cysts segmentations'

#Iterate through all the subfolders in the input folder
for folder in glob(inputFolder+'\*')[1:]:
    #For each of them, create a subfolder in the output folder
    folderName = os.path.basename(folder)

    if not os.path.exists(f'{outputFolder}/{folderName}'):
        os.makedirs(f'{outputFolder}/{folderName}')

    #Iterate through all the kidneys in the subfolde 
    for kidney in glob(folder+'\*'):
        kidneyName = os.path.basename(kidney)

        print(f'Processing kidney {kidneyName} from {folderName}')
        resultMaskPath = f'{outputFolder}/{folderName}/{kidneyName}'
        if not os.path.exists(resultMaskPath):
            os.makedirs(resultMaskPath)

        #Apply morphological opening and save a new mask
        print('Applying morphological opening...')
        morphoProcessingGPU(kidney,resultMaskPath,strRad = 10, operation = 'open', imDimension = '2D', processDimension = '3D', startingSlice=0)
        #Connect the components, rewriting the mask
        print('Connecting components...')
        connectComponents(resultMaskPath,resultMaskPath)                                                            


In [ ]:
#Loop through all the connected cysts masks and extract centroids and volumes of their connected components.

from measureConComps import *
from glob import glob
import os

inputFolder = 'D:/0 - Thesis Data/Cysts segmentations'


#Iterate through all the subfolders in the input folder
for folder in glob(inputFolder+'\*'):
    #For each of them, create a subfolder in the output folder
    folderName = os.path.basename(folder)

    #Iterate through all the kidneys in the subfolder
    for kidney in glob(folder+'\*'):
        kidneyName = os.path.basename(kidney)

        print(f'Processing kidney {kidneyName} from {folderName}')  

        #Extract centroids and volumes
        print('Extracting centroids and volumes...')
        measureConComps(kidney,folder,startingSlice=0,dimension='2D',typeOfMask='cysts',saveSlices = False)
        